# Introdução
Bem-vindo ao projeto de construção de um notebook de web scraping focado em adquirir dados do Sofascore, especificamente dados gerais do Campeonato Brasileiro (Brasileirão). Vamos adentrar no estudo sobre extração e manipulação de dados e análise de informações relevantes no contexto esportivo.

O web scraping é uma técnica de extração automatizada de dados de websites. É uma habilidade essencial para engenheiros de dados, cientistas de dados e desenvolvedores de machine learning, pois permite coletar grandes volumes de dados de fontes públicas para análise e modelagem. No caso do Sofascore, um site conhecido por fornecer estatísticas detalhadas e atualizadas sobre diversos esportes, os dados podem ser usados para análises preditivas, estudos de desempenho, visualizações e muito mais.

<div style="text-align: center;">
  <img src="https://images.unsplash.com/photo-1624880357913-a8539238245b?q=80&w=1000&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D" width="70%">
</div>


> OBJETIVO: Criar um script capaz de adquirir os dados da *tabela de classificação* do campeonato brasileiro de 2023 do site de estatística esportiva *Sofascore*.

In [24]:
import requests
import pandas as pd
import datetime
import duckdb

Web scraping é uma técnica poderosa para coletar dados de websites, mas nem sempre é a melhor abordagem, especialmente quando o site disponibiliza APIs que permitem o acesso direto aos dados. Usar APIs em vez de bibliotecas como BeautifulSoup por exemplo pode oferecer várias vantagens como:

**Facilidade e Eficiência:** APIs são projetadas para fornecer dados de maneira estruturada e eficiente. Com uma chamada de API, você pode obter exatamente os dados que precisa sem ter que navegar pela estrutura HTML da página.

**Confiabilidade:** A estrutura das páginas web pode mudar frequentemente, o que pode quebrar seu script de web scraping. As APIs, por outro lado, têm versões controladas e são mais estáveis, garantindo que seus scripts continuem funcionando mesmo após atualizações no site.

Assim, vamos utilizar as APIs que o site fornece para acessar esses dados diretamente.

Ao explorar a aba de rede nas ferramentas do desenvolvedor, podemos identificar as chamadas de API que o site realiza, permitindo-nos replicar essas solicitações em nosso próprio código.

In [25]:
# These are the headers we need to access the API
headers = {
    'authority': 'api.sofascore.com',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'dnt': '1',
    'if-none-match': 'W/"4bebed6144"',
    'origin': 'https://www.sofascore.com',
    'referer': 'https://www.sofascore.com/',
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
}

# URL da API do Sofascore
url = 'https://www.sofascore.com/api/v1/unique-tournament/325/season/58766/standings/total'

Passo a Passo para pegar os dados que você deseja do site via API:

- 1: Encontre o mapa de chamada da API na guia rede. Clique botão direito do mouse na página web, vá em inspecionar. Após abrir uma pagina no seu navegador, clique em network e logo abaixo em Fetch/XHR e de um refresh na página que você quer pegar os dados;
- 2: Clique com o botão direito e copie como cURL;
- 3: Entre no site curlconverter.com e cole o cURL;
- 4: copie o código python e introduza em seu trabalho.

URL da API e Cabeçalhos: A URL da API é especificada e cabeçalhos são incluídos para emular um navegador real, o que pode ser necessário para evitar bloqueios por parte do servidor.

Função get_sofascore_data: Esta função encapsula a lógica para fazer a solicitação HTTP e lidar com possíveis erros.

In [26]:
# Função para obter dados da API do Sofascore
def get_sofascore_data(url, headers=None):
    try:
        # Fazer a solicitação HTTP
        response = requests.get(url, headers=headers)
        
        # Verificar se a solicitação foi bem-sucedida
        response.raise_for_status()  # Isso levantará um erro para códigos de status 4xx/5xx
        
        # Retornar os dados JSON
        return response.json()
    
    except requests.exceptions.HTTPError as http_err:
        print(f"Erro HTTP ao acessar {url}: {http_err}")
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Erro de conexão ao acessar {url}: {conn_err}")
    except requests.exceptions.Timeout as timeout_err:
        print(f"Tempo de solicitação esgotado ao acessar {url}: {timeout_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Erro ao acessar {url}: {req_err}")
    except ValueError as json_err:
        print(f"Erro ao analisar JSON de {url}: {json_err}")
    
    # Retornar None em caso de erro
    return None

In [27]:
# Obter dados da API
data = get_sofascore_data(url, headers)

if data:
    # Processar os dados JSON conforme necessário
    print("Dados obtidos com sucesso!")
else:
    print("Falha ao obter dados.")

Dados obtidos com sucesso!


In [28]:
data

{'standings': [{'tournament': {'name': 'Brasileirão Série A',
    'slug': 'brasileirao-serie-a',
    'category': {'name': 'Brazil',
     'slug': 'brazil',
     'sport': {'name': 'Football', 'slug': 'football', 'id': 1},
     'id': 13,
     'flag': 'brazil',
     'alpha2': 'BR'},
    'uniqueTournament': {'name': 'Brasileirão Série A',
     'slug': 'brasileirao-serie-a',
     'primaryColorHex': '#C7FF00',
     'secondaryColorHex': '#969696',
     'category': {'name': 'Brazil',
      'slug': 'brazil',
      'sport': {'name': 'Football', 'slug': 'football', 'id': 1},
      'id': 13,
      'flag': 'brazil',
      'alpha2': 'BR'},
     'userCount': 344626,
     'hasPerformanceGraphFeature': True,
     'id': 325,
     'displayInverseHomeAwayTeams': False},
    'priority': 442,
    'isGroup': False,
    'isLive': False,
    'id': 83},
   'type': 'total',
   'name': 'Brasileiro Serie A',
   'descriptions': [],
   'tieBreakingRule': {'text': 'In the event that two (or more) teams have an equal n

Verificando todas as chaves do JSON

In [29]:
data.keys()

dict_keys(['standings'])

In [30]:
len(data.get('standings'))

1

In [31]:
data.get('standings')[0].keys()

dict_keys(['tournament', 'type', 'name', 'descriptions', 'tieBreakingRule', 'rows', 'id', 'updatedAtTimestamp'])

In [32]:
data.get('standings')[0].get('tournament')

{'name': 'Brasileirão Série A',
 'slug': 'brasileirao-serie-a',
 'category': {'name': 'Brazil',
  'slug': 'brazil',
  'sport': {'name': 'Football', 'slug': 'football', 'id': 1},
  'id': 13,
  'flag': 'brazil',
  'alpha2': 'BR'},
 'uniqueTournament': {'name': 'Brasileirão Série A',
  'slug': 'brasileirao-serie-a',
  'primaryColorHex': '#C7FF00',
  'secondaryColorHex': '#969696',
  'category': {'name': 'Brazil',
   'slug': 'brazil',
   'sport': {'name': 'Football', 'slug': 'football', 'id': 1},
   'id': 13,
   'flag': 'brazil',
   'alpha2': 'BR'},
  'userCount': 344626,
  'hasPerformanceGraphFeature': True,
  'id': 325,
  'displayInverseHomeAwayTeams': False},
 'priority': 442,
 'isGroup': False,
 'isLive': False,
 'id': 83}

In [33]:
data.get('standings')[0].get('type')

'total'

In [34]:
data.get('standings')[0].get('name')

'Brasileiro Serie A'

In [35]:
data.get('standings')[0].get('descriptions')

[]

In [36]:
data.get('standings')[0].get('tieBreakingRule')

{'text': 'In the event that two (or more) teams have an equal number of points, the following rules break the tie:\n\n1. Number of victories\n2. Goal difference\n3. Goals scored',
 'id': 1347}

In [37]:
data.get('standings')[0].get('id')

119336

In [38]:
data.get('standings')[0].get('updatedAtTimestamp')

1713065191

In [39]:
data.get('standings')[0].get('rows')[0].keys()

dict_keys(['team', 'descriptions', 'promotion', 'position', 'matches', 'wins', 'scoresFor', 'scoresAgainst', 'id', 'losses', 'draws', 'points'])

# Manipulando dados do JSON

Nesse momento, vamos criar um dataframe com informações do campeonato analisado

In [40]:
# Extraindo informações do campeonato
tournament_info = data['standings'][0]['tournament']
tournament_df = pd.DataFrame([{
    'id': tournament_info['uniqueTournament']['id'],
    'name': tournament_info['name'],
    'slug': tournament_info['slug'],
    'updatedAtTimestamp': datetime.datetime.fromtimestamp(data['standings'][0]['updatedAtTimestamp'])
}])

tournament_df

,id,name,slug,updatedAtTimestamp
0,325,Brasileirão Série A,brasileirao-serie-a,2024-04-14 00:26:31


Nota-se que o ID é referente ao campeonato brasileiro. Será importante ter um Identificador único, pois caso precisarmos relacionar jogos, dados de jogadores, estatisticas gerais com o campeonato brasileiro, será muito mais fácil. Sempre bom termos ID em nossas tabelas.

Agora vamos construir a base de classificação do campeonato brasileiro e atribuir a um dataframe

In [41]:
# Extraindo os dados necessários
standings = data['standings'][0]['rows']
extracted_data = []

for row in standings:
    extracted_data.append({
        'id_uniqueTournament': tournament_info['uniqueTournament']['id'],
        'position': row['position'],
        'id': row['id'],
        'team': row['team']['name'],
        'matches': row['matches'],
        'scoresFor': row['scoresFor'],
        'scoresAgainst': row['scoresAgainst'],
        'wins': row['wins'],
        'losses': row['losses'],
        'draws': row['draws'],
        'points': row['points']
    })

# Criando o DataFrame
df = pd.DataFrame(extracted_data)
df

,id_uniqueTournament,position,id,team,matches,scoresFor,scoresAgainst,wins,losses,draws,points
0,325,1,1036711,Athletico,6,9,3,4,1,1,13
1,325,2,1036717,Bahia,6,9,6,4,1,1,13
2,325,3,1036712,Flamengo,6,7,5,3,1,2,11
3,325,4,1036710,Botafogo,6,12,7,3,2,1,10
4,325,5,1036724,São Paulo,6,10,6,3,2,1,10
5,325,6,1036715,Cruzeiro,5,8,7,3,1,1,10
6,325,7,1036709,Atlético Mineiro,5,9,3,2,0,3,9
7,325,8,1036723,Red Bull Bragantino,6,7,6,2,1,3,9
8,325,9,1036727,Palmeiras,6,3,3,2,2,2,8
9,325,10,1036726,Internacional,4,4,3,2,1,1,7


Note que novamente atibuímos o ID do campeonato brasileiro a essa base e pegamos um ID muito importante: dos times do campeonato. Será muito importante para possíveis relacionamentos entre tabelas.

# Integração com Banco de Dados do DuckDB

Colocar dados em um banco de dados como o DuckDB proporciona organização, segurança e eficiência na manipulação e análise de grandes volumes de dados, sendo uma escolha adequada especialmente para aplicações que necessitam de consultas rápidas e análises complexas.

Um banco de dados permite armazenar e organizar grandes volumes de dados de forma estruturada. Isso facilita o gerenciamento, a consulta e a atualização dos dados conforme necessário, sem comprometer o desempenho.

Assim, quando precisarmos relacionar informações para um possível treinamento de modelo de machine learning, deep learning, ou até mesmo para Data Analytics, já iremos ter um banco de dados com todas as informações disponíveis.

## Por que escolher DuckDB:
- Desempenho otimizado para análises rápidas: DuckDB é conhecido por seu desempenho otimizado para consultas analíticas e operações de leitura, especialmente em conjuntos de dados maiores. Ele é projetado para ser rápido mesmo em ambientes onde o tempo de resposta é crítico;

- Compatibilidade com SQL padrão: DuckDB oferece suporte ao SQL padrão, o que facilita a transição e o uso por desenvolvedores familiarizados com bancos de dados relacionais tradicionais. Isso permite aproveitar habilidades existentes em SQL para manipular e analisar dados;



In [42]:
# Criando conexão com o banco de dados DuckDB local
con = duckdb.connect('database/BrasilStatsDB.db')

# Inserindo o dataframe do campeonato na tabela 'tournament'
con.execute('CREATE OR REPLACE TABLE descricao_campeonatos AS SELECT * FROM tournament_df')

# Inserindo o dataframe das equipes na tabela 'teams'
con.execute('CREATE OR REPLACE TABLE campeonato_brasileiro AS SELECT * FROM df')

Verificando se as tabelas foram criadas corretamente

In [43]:
query = 'SELECT * FROM descricao_campeonatos'
result = con.execute(query).fetchdf()
result

,id,name,slug,updatedAtTimestamp
0,325,Brasileirão Série A,brasileirao-serie-a,2024-04-14 00:26:31


In [44]:
query = 'SELECT * FROM campeonato_brasileiro WHERE position = 1'
result = con.execute(query).fetchdf()
result

,id_uniqueTournament,position,id,team,matches,scoresFor,scoresAgainst,wins,losses,draws,points
0,325,1,1036711,Athletico,6,9,3,4,1,1,13


# Conclusão

Usar APIs para coletar dados é uma abordagem poderosa que oferece várias vantagens em comparação com o web scraping tradicional. Ao utilizar as APIs do Sofascore, garantimos que nossos dados são extraídos de maneira eficiente, confiável. Para futuros projetos, considere explorar outras APIs esportivas e integrar diferentes fontes de dados para análises mais abrangentes.

Espero que este guia tenha sido útil e claro. Boa sorte com o seu projeto de web scraping e continue expandindo suas habilidades em ciência de dados e machine learning!